# Deep Learning based FACs and Action Unit occurrence and Intensity Estimation

#### \- Amogh Gudi, Emrah Tasli, Tim den Uyl, Andreas Marulis
https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7284873

### Plan
1. Load dataset and labels
2. Build a model
3. Train Model
4. Test Model

**Importing Libraries**

In [1]:
import keras
import numpy as np

/home/amogh/anaconda3/envs/faus_dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

### 1. Loading dataset and labels

a. Loading dataset

In [4]:
BP4D_base_path='/home/amogh/cmu/dataset/BP4D/'

In [5]:
BP4D_training_folder=BP4D_base_path+'BP4D-training/'

In [6]:
from keras.preprocessing.image import ImageDataGenerator

In [7]:
gen=ImageDataGenerator()

Batches from directory:

In [8]:
batch_size=32
batches_from_dir=gen.flow_from_directory(BP4D_training_folder,target_size=(48,48),batch_size=batch_size)

Found 553 images belonging to 1 classes.


b. Loading labels

In [9]:
BP4D_AU_path=BP4D_base_path+'AUCoding/AUCoding/'

Function to get labels according to subject, sequence, frame No: (details and examples in datasets.ipynb)

In [10]:
def label_getter(subj_req,seq_req,frame_req):
    au_file_reqd=BP4D_AU_path+f'{subj_req}_{seq_req}.csv'
#     print (au_file_reqd)
    df_reqd=pd.read_csv(au_file_reqd)
    list_of_faus=list(df_reqd.loc[df_reqd['0']==frame_req].iloc[0]) #list: [frame_no, fau1, fau2, fau3....]
    #choose FAUS for which you want the labels.
    list_mask=[1,2,4,6,7,10,12,14,15,17,23]
    list_final=[list_of_faus[i] for i in list_mask]
    return list_final

In [11]:
label_getter('F001','T1',2440)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]

We define our own flow_from_directory function so that we can assign multiple labels to the training data before passing it to the training function

In [12]:
file_names=batches_from_dir.filenames
file_names

['F001/T1/2440.jpg',
 'F001/T1/2441.jpg',
 'F001/T1/2442.jpg',
 'F001/T1/2443.jpg',
 'F001/T1/2444.jpg',
 'F001/T1/2445.jpg',
 'F001/T1/2446.jpg',
 'F001/T1/2447.jpg',
 'F001/T1/2448.jpg',
 'F001/T1/2449.jpg',
 'F001/T1/2450.jpg',
 'F001/T1/2451.jpg',
 'F001/T1/2452.jpg',
 'F001/T1/2453.jpg',
 'F001/T1/2454.jpg',
 'F001/T1/2455.jpg',
 'F001/T1/2456.jpg',
 'F001/T1/2457.jpg',
 'F001/T1/2458.jpg',
 'F001/T1/2459.jpg',
 'F001/T1/2460.jpg',
 'F001/T1/2461.jpg',
 'F001/T1/2462.jpg',
 'F001/T1/2463.jpg',
 'F001/T1/2464.jpg',
 'F001/T1/2465.jpg',
 'F001/T1/2466.jpg',
 'F001/T1/2467.jpg',
 'F001/T1/2468.jpg',
 'F001/T1/2469.jpg',
 'F001/T1/2470.jpg',
 'F001/T1/2471.jpg',
 'F001/T1/2472.jpg',
 'F001/T1/2473.jpg',
 'F001/T1/2474.jpg',
 'F001/T1/2475.jpg',
 'F001/T1/2476.jpg',
 'F001/T1/2477.jpg',
 'F001/T1/2478.jpg',
 'F001/T1/2479.jpg',
 'F001/T1/2480.jpg',
 'F001/T1/2481.jpg',
 'F001/T1/2482.jpg',
 'F001/T1/2483.jpg',
 'F001/T1/2484.jpg',
 'F001/T1/2485.jpg',
 'F001/T1/2486.jpg',
 'F001/T1/248

In [37]:
def multiclass_flow_from_directory(batches_from_dir, label_getter):
    batch_count=0
    for x, y in batches_from_dir:
        print ('batch number is: ',batch_count, ' file indexes from ', batch_size*batch_count,' and ', batch_size*batch_count+batch_size-1)
        first_image_index=batch_size*batch_count
        last_image_index=batch_size*batch_count+batch_size-1
        fau_labels=[(file.split('.')[0].split('/')[0],file.split('.')[0].split('/')[1],int(file.split('.')[0].split('/')[2])) for file in file_names[first_image_index:last_image_index+1] ]
        print (fau_labels)
        batch_count+=1
        yield x, fau_labels

In [38]:
generator_with_modified_labels=multiclass_flow_from_directory(batches_from_dir,label_getter)

In [39]:
for i in gen:
    idx = (gen.batch_index - 1) * gen.batch_size
    print(gen.filenames[idx : idx + gen.batch_size])

TypeError: 'ImageDataGenerator' object is not iterable

In [40]:
count2=0
for obj in generator_with_modified_labels:
#     if count2<2:
#         print (1, len(obj[0]))
    print ('newer gen being iterated these many times: ',count2)
    count2+=1

batch number is:  0  file indexes from  0  and  31
[('F001', 'T1', 2440), ('F001', 'T1', 2441), ('F001', 'T1', 2442), ('F001', 'T1', 2443), ('F001', 'T1', 2444), ('F001', 'T1', 2445), ('F001', 'T1', 2446), ('F001', 'T1', 2447), ('F001', 'T1', 2448), ('F001', 'T1', 2449), ('F001', 'T1', 2450), ('F001', 'T1', 2451), ('F001', 'T1', 2452), ('F001', 'T1', 2453), ('F001', 'T1', 2454), ('F001', 'T1', 2455), ('F001', 'T1', 2456), ('F001', 'T1', 2457), ('F001', 'T1', 2458), ('F001', 'T1', 2459), ('F001', 'T1', 2460), ('F001', 'T1', 2461), ('F001', 'T1', 2462), ('F001', 'T1', 2463), ('F001', 'T1', 2464), ('F001', 'T1', 2465), ('F001', 'T1', 2466), ('F001', 'T1', 2467), ('F001', 'T1', 2468), ('F001', 'T1', 2469), ('F001', 'T1', 2470), ('F001', 'T1', 2471)]
newer gen being iterated these many times:  0
batch number is:  1  file indexes from  32  and  63
[('F001', 'T1', 2472), ('F001', 'T1', 2473), ('F001', 'T1', 2474), ('F001', 'T1', 2475), ('F001', 'T1', 2476), ('F001', 'T1', 2477), ('F001', 'T1'

KeyboardInterrupt: 

In [ ]:
BP4D_training_folder+batches_from_dir.filenames[23]

In [ ]:
plt.imread(BP4D_training_folder+batches_from_dir.filenames[23])

In [43]:
keras.utils.to_categorical((3,4,4,1),6)

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.]])

### 2. Building a Model

In [38]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D,Flatten,Dropout
from keras.optimizers import SGD

In [34]:
model=Sequential()
model.add(Conv2D(64,kernel_size=(5,5),strides=(1,1),activation='relu',input_shape=(48,48,1)))
model.add(MaxPool2D(pool_size=(3,3),strides=(2,2)))
model.add(Conv2D(64,kernel_size=(5,5),strides=(1,1),activation='relu'))
model.add(Conv2D(128,kernel_size=(4,4),strides=(1,1),activation='relu'))
model.add(Flatten())
model.add(Dense(3072,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6,activation='sigmoid'))

In [42]:
model.compile(loss='binary_crossentropy',optimizer=SGD(lr=0.01,momentum=0.9))

In [43]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 44, 44, 64)        1664      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 17, 17, 64)        102464    
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 14, 14, 128)       131200    
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 3072)              77073408  
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 18438     
Total para

### 3. Training the model

### 4. Testing the model